In [1]:
"""
This is the main module which uses GUI and makes the menu.
NOTE - the bar chart is a 3 plot graph that because of compatability issues 
with Matplot and IPYWidgets will open in a new window. If not, it should do, so please contact me.
Student ID: F313427
"""

import database as d
import gameSearch as g
import gameRent as r
import gameReturn as re
import feedbackManager as fm
import InventoryPruning as ip
import ipywidgets as widgets
from IPython.display import display, HTML
from IPython.display import clear_output

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties



'''
Description:
    This function completes the pruning process. 
    It compares each game to certain conditions as described in the pruning module. 
Returns/Output:
    Prints a a list of games to remove and a bar chart with 3 subplots. 
'''
def suggest_removal():
    prune_suggest = []

    rent_num = []
    feedbacks = []
    date_diff = []
    with open('Game_Info.txt', 'r') as games:
        individual_line = games.readlines()
        for line in individual_line:
            line_split = line.strip().split(',')
            game_ID = line_split[0]

            rent_count = ip.number_of_rents(game_ID)
            feedback_avg = ip.game_feedback(game_ID)
            date_difference = ip.game_date_difference(game_ID)

            if rent_count <= 3 and feedback_avg <= 2 and date_difference > 25:
                prune_suggest.append(game_ID)

                rent_num.append(rent_count)
                feedbacks.append(feedback_avg)
                date_diff.append(date_difference)

    print("Suggested removal of the following games: ")
    print(prune_suggest)

    font = FontProperties()
    font.set_family('serif')
    font.set_name('Times New Roman')
    font.set_style('oblique')

    plt.figure(figsize=(12, 8))
    plt.style.use('ggplot')

    plt.subplot(2, 2, 1)
    plt.bar(prune_suggest, rent_num, color='green', label='Number of Rents')
    plt.legend()
    plt.grid(color='black', linestyle='--', linewidth='0.3')
    plt.title("Number of Rents", color='black', fontsize='x-large', weight='bold')
    plt.xlabel("Suggested Pruning Game ID", color='black', fontsize=15, fontproperties=font)
    plt.ylabel("Number of Rent", color='black', fontsize=15, fontproperties=font)
    plt.ylim(0, 3)

    plt.subplot(2, 2, 2)
    plt.bar(prune_suggest, feedbacks, color='green', label='Average Feedback')
    plt.legend()
    plt.grid(color='black', linestyle='--', linewidth='0.3')
    plt.title("Average Feedback", color='black', fontsize='x-large', weight='bold')
    plt.xlabel("Suggested Pruning Game ID", color='black', fontsize=15, fontproperties=font)
    plt.ylabel("Avg Feedback", color='black', fontsize=15, fontproperties=font)
    plt.ylim(0, 5)

    plt.subplot(2, 2, 3)
    plt.bar(prune_suggest, date_diff, color='green', label='Last Rent Date')
    plt.legend()
    plt.grid(color='black', linestyle='--', linewidth='0.3')
    plt.title("Date Difference", color='black', fontsize='x-large', weight='bold')
    plt.xlabel("Suggested Pruning Game ID", color='black', fontsize=15, fontproperties=font)
    plt.ylabel("Days from Last Rent", color='black', fontsize=15, fontproperties=font)

    plt.subplots_adjust(hspace=0.3)
    plt.show(block=True)


#GUI Functions
#Output widgets dictionary
output_widgets = {
    'search': widgets.Output(),
    'rent': widgets.Output(),
    'return': widgets.Output(),
    'feedback': widgets.Output(),
    'prune': widgets.Output()
}

#reset button to remove all entries on click - including print statements
def on_reset_btn_click(b, feature):
    output_widget = output_widgets.get(feature)
    if output_widget:
        with output_widget:
            clear_output()


'''
Description:
    This completes the search process - it asks the user if they'd like to search by title, genre or platform.
Parameters:
    b - button - this function is ran when a button on the GUI is pressed
Returns/Output:
    Outputs a list of games applicable to the search criteria including the availability
'''
def on_search_btn_click(b):
    search_choice_strip = search_choice.value.strip()

    if search_choice_strip == "Title":
        search_chosen_strip = search_chosen.value.strip()
        with output_widgets['search']:
            clear_output()
            g.search(search_chosen_strip, 3)
    elif search_choice_strip == "Genre":
        search_chosen_strip = search_chosen.value.strip()
        with output_widgets['search']:
            clear_output()
            g.search(search_chosen_strip, 2)
    elif search_choice_strip == "Platform":
        search_chosen_strip = search_chosen.value.strip()
        with output_widgets['search']:
            clear_output()
            g.search(search_chosen_strip, 1)
    search_chosen.value = ""




'''
Description:
    This completes the rent process. It allows the user to rent a singular game, 
    or more than one game at a time. It performs checks such as customer validity and availability.
Parameters:
    b - button - this function is ran when a button on the GUI is pressed
Returns/Output:
    Adds a line to the Rental text file + prints a message to the user. 
'''
def on_rent_btn_click(b):
    rent_customer_strip = rent_customer.value.strip()
    with output_widgets['rent']:
        clear_output()
        if d.validate_customer_ID_format(rent_customer_strip) and d.customer_exists(rent_customer_strip):
            #print(rent_customer_strip)
            if "," in rent_game.value:
                split_games = rent_game.value.split(",")
                for game in split_games:
                    strip_game = game.strip()
                    if d.game_exists(strip_game):
                        if d.availability(strip_game):
                            r.rent_game(rent_customer_strip, strip_game)
                        else:
                            print("Game " + strip_game + " Unavailable")
                rent_customer.value = ""
                rent_game.value = ""
            else:
                strip_game = rent_game.value.strip()
                #print(strip_game)
                if d.game_exists(strip_game):
                    if d.availability(strip_game):
                        r.rent_game(rent_customer_strip, strip_game)
                        rent_customer.value = ""
                        rent_game.value = ""
                    else:
                        print("Game " + strip_game + " Unavailable")



'''
Description:
    This completes the return process. It only allows the user to input one game to return 
    at a time, not more. This is because they will be asked for feedback at the same time. 
    This function doesn't deal with the addition of feedback, but gives the user the option
    to add feedback (through the checkbox).
Parameters:
    b - button - this function is ran when a button on the GUI is pressed
Returns/Output:
    For a given game, it adds a return date in the Rental text file and prints a message to the user. 
'''
def on_return_btn_click(b):
    with output_widgets['return']:
        clear_output()
        if "," in return_game.value:
            split_games = return_game.value.split(",")
            for game in split_games:
                strip_game = game.strip()
                if d.game_exists(strip_game):
                    if not d.availability(strip_game):
                        re.return_game(strip_game)
                        if option_feedback.value:
                            feedback_form = create_feedback_form(strip_game)
                            display(feedback_form)
                    else:
                        print(f"Game {strip_game} Already Available")
        else:
            strip_game = return_game.value.strip()
            if d.game_exists(strip_game):
                if not d.availability(strip_game):
                    re.return_game(strip_game)
                    if option_feedback.value:
                        feedback_form = create_feedback_form(strip_game)
                        display(feedback_form)
                else:
                    print(f"Game {strip_game} Already Available")


feedback_forms = []

def create_feedback_form(game_id):
    #Feedback Widgets
    rating = widgets.IntSlider(min=0, max=5, value=2, step=1, description='Rating: ', orientation='horizontal',
                               readout=True, readout_format='d')
    comment = widgets.Dropdown(description='Comment: ', placeholder='Choose Option',
                               options=['Excellent Game!', 'Highly Enjoyable!', 'Great but could be better!',
                                        'Average experience.', 'Not very engaging.', 'Boring.', 'Rubbish experience.'])
    feedback_button = widgets.Button(description='Submit', button_style='info', tooltip='Click Me')

    # Create a dictionary to store feedback form components and their associated game ID
    feedback_form = widgets.HBox([rating, comment, feedback_button])
    feedback_forms.append(feedback_form)

    feedback_button.on_click(lambda b: submit_feedback(feedback_form, game_id))
    return feedback_form

def submit_feedback(feedback_form, game_id):
    feedback_forms.remove(feedback_form)
        
    rating = feedback_form.children[0].value
    comment = feedback_form.children[1].value
    
    fm.add_feedback(game_id, int(rating), comment, 'Game_Feedback.txt')
    print(f"Feedback for {game_id} Added")
    with output_widgets['return']:
        feedback_form.layout.display = 'none'
        return_game.value = ""


'''
Description:
    This function completes the prune process. It suggests games the user should remove
    and display a bar graph with 3 subplots. 
Parameters:
    b - button - this function is ran when a button on the GUI is pressed
Returns/Output:
    Prints a list of suggested games to the user, and a bar graph to the user. 
'''
def on_prune_btn_click(b):
    with output_widgets['prune']:
        clear_output()
        suggest_removal()

custom_button_css = """
<style>
    .widget-button {
        background-color: grey !important;
        color: white;
        border-radius: 3px;
    }
    
    .p-Accordion-child > .p-Collapse-header{
    background-color: grey;   
    border: 1px solid black; 
    color: black;
    }
    
</style>
"""


display(HTML(custom_button_css))

#Search Widgets
search_choice = widgets.Dropdown(options=['Title', 'Genre', 'Platform'], description='Search by: ', )
search_chosen = widgets.Text(description='Search for: ', placeholder='Enter Search')
search_button = widgets.Button(description='Search', button_style='info', tooltip='Click Me')
reset_search_button = widgets.Button(description='Reset Search', button_style='info', tooltip='Click Me')

#Rent Widgets
rent_customer = widgets.Text(description='Customer ID: ', placeholder='Enter Customer ID')
rent_game = widgets.Text(description='Game ID(s): ', placeholder='Enter Game ID(s)')
rent_button = widgets.Button(description='Rent', button_style='info', tooltip='Click Me')
reset_rent_button = widgets.Button(description='Reset Rent', button_style='info', tooltip='Click Me')

#Return Widgets
return_game = widgets.Text(description='Game ID: ', placeholder='Enter Game ID')
option_feedback = widgets.Checkbox(description="Add Feedback")
return_button = widgets.Button(description='Return', button_style='info', tooltip='Click Me')
reset_return_button = widgets.Button(description='Reset Return', button_style='info', tooltip='Click Me')


#Prune Widgets
prune_button = widgets.Button(description='Prune', button_style='info', tooltip='Click Me')
label = widgets.HTML(value='<b>Click here to prune<b>', layout=widgets.Layout(width='70%' ))
reset_prune_button = widgets.Button(description='Reset Prune', button_style='info', tooltip='Click Me')

search_button.on_click(on_search_btn_click)
rent_button.on_click(on_rent_btn_click)
return_button.on_click(on_return_btn_click)
prune_button.on_click(on_prune_btn_click)

reset_search_button.on_click(lambda b: on_reset_btn_click(b, 'search'))
reset_rent_button.on_click(lambda b: on_reset_btn_click(b, 'rent'))
reset_return_button.on_click(lambda b: on_reset_btn_click(b, 'return'))
reset_prune_button.on_click(lambda b: on_reset_btn_click(b, 'prune'))

#centre display for buttons
center_layout = widgets.Layout(display='flex', justify_content='center', width='100%', margin='20px 0 0 0 ')

#Widget Layout
search_inputs = widgets.HBox([search_choice, search_chosen], layout=center_layout)
search_buttons = widgets.HBox([search_button, reset_search_button], layout=center_layout)

rent_inputs = widgets.HBox([rent_customer, rent_game], layout=center_layout)
rent_buttons = widgets.HBox([rent_button, reset_rent_button], layout=center_layout)

return_inputs = widgets.HBox([return_game, option_feedback], layout=center_layout)
return_buttons = widgets.HBox([return_button, reset_return_button], layout=center_layout)

prune_buttons = widgets.HBox([label, prune_button, reset_prune_button])

#Output Layouts
search_display = widgets.VBox([search_inputs, search_buttons, output_widgets['search']])
rent_display = widgets.VBox([rent_inputs, rent_buttons, output_widgets['rent']])
return_display = widgets.VBox([return_inputs, return_buttons, output_widgets['return']])
prune_display = widgets.VBox([prune_buttons, output_widgets['prune']])

#Accordian
accordian= widgets.Accordion(children=[search_display, rent_display, return_display, prune_display], titles=('Search Options','Rent Games','Return Games','Inventory Pruning'))

# Display the tab
display(accordian)

Accordion(children=(VBox(children=(HBox(children=(Dropdown(description='Search by: ', options=('Title', 'Genre…